In [1]:
import sys 
sys.path.append('../')
sys.path.append('../../')

from utils.logging_utils import * 
from datasets import dataset_factory 
from datasets.dataset_utils import * 
from utils.common_utils import * 
from core.passive_learning import * 

# configuration
config_file ='../conf/basic-ml/MNIST_lenet.yaml'
#conf = load_yaml_config(config_file)
conf = load_yaml_config_std(config_file)
conf['base_dir'] = '../'
conf = OmegaConf.create(conf)
    
logger = get_logger(conf.output.log_file_path,stdout_redirect=True,level=logging.DEBUG)

In [2]:
conf

{'device': 'cuda:0', 'random_seed': 1, 'data_conf': {'dataset': 'mnist', 'dimension': 784, 'random_state': 0, 'data_path': '../../datapath/', 'val_fraction': 0.3, 'flatten': False}, 'model_conf': {'model_name': 'lenet', 'num_classes': 10, 'fit_intercept': True, 'lib': 'pytorch'}, 'training_conf': {'optimizer': 'sgd', 'learning_rate': 0.1, 'loss_tolerance': 1e-06, 'max_epochs': 20, 'normalize_weights': False, 'batch_size': 128, 'log_batch_loss_freq': 100, 'ckpt_load_path': None}, 'inference_conf': {'shuffle': False, 'batch_size': 512}, 'output': {'log_file_path': '${base_dir}/outputs/logs/temp.log'}, 'base_dir': '../'}

In [3]:

set_seed(conf.random_seed)
# get data
ds = dataset_factory.load_dataset(conf)
ds.build_dataset()
train_set,val_set_std = randomly_split_dataset(ds,1- conf.data_conf.val_fraction)
# this val_set is the fixed val_set of big size. the algo will be given a subsample of it.

test_ds = ds.get_test_datasets()

pas_learn = PassiveLearning(conf, train_set, ds_val=val_set_std,ds_test=test_ds,logger=logger)
out = pas_learn.run()
    

[12/13/2022 12:46:56 PM MyLogger : INFO  : passive_le : run ] : Train data size: 42000
[12/13/2022 12:46:56 PM MyLogger : INFO  : passive_le : run ] : --------------- Begin Model Training ------------
[12/13/2022 12:46:56 PM MyLogger : INFO  : passive_le : run ] : Training conf :{'optimizer': 'sgd', 'learning_rate': 0.1, 'loss_tolerance': 1e-06, 'max_epochs': 20, 'normalize_weights': False, 'batch_size': 128, 'log_batch_loss_freq': 100, 'ckpt_load_path': None, 'device': 'cuda:0', 'ckpt_save_path': None, 'embedding_save_path': None}
[12/13/2022 12:46:56 PM MyLogger : INFO  : passive_le : run ] : Model conf : {'model_name': 'lenet', 'num_classes': 10, 'fit_intercept': True, 'lib': 'pytorch'}
[12/13/2022 12:46:56 PM MyLogger : DEBUG : model_trai : train ] : Training conf : {'optimizer': 'sgd', 'learning_rate': 0.1, 'loss_tolerance': 1e-06, 'max_epochs': 20, 'normalize_weights': False, 'batch_size': 128, 'log_batch_loss_freq': 100, 'ckpt_load_path': None, 'device': 'cuda:0', 'ckpt_save_pat

In [5]:
clf = pas_learn.cur_clf
pas_learn.get_test_error(clf,test_ds,conf['inference_conf'])

0.01429999999999998